### Setup the running environment 
Assuming you have conda install on you system and this is running on your Jupyter Notebook server:
1. create a new environment (HINT: `conda create -n phase-diagram python=3.9`) and activate (`conda activate phase-diagram`)
2. install pymatgen and ase (copy this line to the next empty cell `!conda install -y pymatgen ase`)
With that, you can run the following code!

code creator: Kian Pu. If you have any questions of using this code, please email/slack me. 

In [1]:
from pymatgen.analysis.phase_diagram import PDPlotter,CompoundPhaseDiagram
from pymatgen.entries.computed_entries import ComputedEntry
from pymatgen.io.ase import AseAtomsAdaptor
%matplotlib inline

import pandas
from ase.db import connect

/jet/home/jpu/softwares/envs/gasp-qe/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
database_name = input('What is your database name?')

In [3]:
ase_db = connect(database_name)
AAA=AseAtomsAdaptor()

In [4]:
entries_lst = []
endpoint_lst = []
entry_dict={}
for i,ID in enumerate(range(1,1+len(ase_db))):
    atoms = ase_db.get_atoms(ID)
    entry_id = ase_db.get(ID).entry_id
    struc=AAA.get_structure(atoms)
    composition = struc.composition
    energy = atoms.get_potential_energy()
    correction = 0 
    entry = ComputedEntry(composition, energy, correction,entry_id=entry_id)
    entries_lst.append(entry)
    entry_dict[entry.entry_id] = i
    #remember to change it to your end points index 
    if entry_id.split('.')[1].split('_')[0] in ['1','3']:
        endpoint_lst.append(i)   
    # if entry_id.split('.')[1] in ['B_ref_1','LiF_ref_1','F2_ref_1']:
    #     print(i,composition)

In [ ]:
pd = CompoundPhaseDiagram(entries_lst,
                            [entries_lst[endpoint_lst[0]].composition,
                            entries_lst[endpoint_lst[1]].composition,
                            entries_lst[endpoint_lst[2]].composition]) #this is for tenary 
plotter = PDPlotter(pd,show_unstable=2,ternary_style='3d')
fig=plotter.get_plot()
fig.update_layout(
    plot_bgcolor='black', #change it to other color if your system background is different
    paper_bgcolor='black',
    font=dict(color='white')
)

In [12]:
# Analyze phase diagram!
formula_lst=[]
stable_lst = []
entry_id_lst =[]
e_above_hull_lst = []
decompose_lst =[]

for e in pd.stable_entries:
    formula_lst.append(entries_lst[entry_dict[e.as_dict()['entry_id']]].composition)
    stable_lst.append('True')
    entry_id_lst.append(e.as_dict()['entry_id'])
    e_above_hull_lst.append(0)
    decompose_lst.append('N/A')

for e in pd.unstable_entries:
    decomp, e_above_hull = pd.get_decomp_and_e_above_hull(e)
    pretty_decomp = [("{}:{}".format(entries_lst[entry_dict[e.as_dict()['entry_id']]].composition, k.as_dict()['entry_id']), round(v, 2)) for k, v in decomp.items()]
    formula_lst.append(entries_lst[entry_dict[e.as_dict()['entry_id']]].composition)
    stable_lst.append('false')
    entry_id_lst.append(e.as_dict()['entry_id'])
    e_above_hull_lst.append("%.3f" % e_above_hull)
    decompose_lst.append(pretty_decomp)

analyze_dict={}
analyze_dict['formula'] = formula_lst
analyze_dict['stable'] = stable_lst
analyze_dict['entry_id'] = entry_id_lst
analyze_dict['e_above_hull (eV/atom)'] = e_above_hull_lst
analyze_dict['decomposition'] = decompose_lst

df=pandas.DataFrame.from_dict(analyze_dict)

In [ ]:
df.head()

In [14]:
df.to_csv('pd_report.csv')